<a href="https://colab.research.google.com/github/RafsanJany-44/ACM/blob/master/Voting_Ensamble_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
#data = 'https://raw.githubusercontent.com/RafsanJany-44/Machine-School/main/datasets/HMC_WITH_BIOM.csv'
#data="https://raw.githubusercontent.com/RafsanJany-44/Research-NREM-REM/main/dataset/REM_NREM.csv"
#data="https://raw.githubusercontent.com/RafsanJany-44/Machine-School/main/datasets/EEG_HMC.csv"
data="https://raw.githubusercontent.com/RafsanJany-44/Machine-School/main/datasets/REM_NREM_O2.csv"
dataset = pd.read_csv(data)

In [ ]:
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 10)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)